# Onbereikbaarheid door wegwerkzaamheden
- De kern van deze casus is de (on)bereikbaarheidsquery die elders al gebruikt wordt. Die query is aangepast om in functievorm gebruikt te worden (en als zodanig vaak/repetitief gerund te worden). 
- Werkzaamheden uit Melvin worden ingeladen en getransformeerd naar 'virtuele verkeersborden', dit is een manier om per wegsegment aan te kunnen geven wat de tijdelijke beperkingen zijn die gelden tijdens de wegwerkzaamheden. Een bijkomend voordeel is dat deze informatie erg makkelijk is samen te voegen met de echte verkeersborden, die als input gelden voor de (on)bereikbaarheidsquery. Door die samenvoeging kan de (on)bereikbaarheidsquery zeer eenvoudig worden gerund voor alle verschillende situaties die gecreeerd worden door de wegwerkzaamheden.
- Voor elke combinatie van wegwerkzaamheid en voertuigtype wordt alleen de aanpassing van de bereikbaarheid ten opzichte van de standaardsituatie weergegeven. De standaardsituatie (per voertuigtype) wordt los opgeslagen. Dit wordt gedaan als optimalisatie.

Nog niet geimplementeerd
- Het omzetten van Melvin data naar 'virtuele verkeersborden' incl. alle beperkingen en mogelijkheden (nu is de aanname dat de weg altijd volledig dicht is)

In [44]:
import psycopg2
try:
    postgres_pw = TokenLibrary.getSecret("redacted", "redacted", "redacted")
    conn = psycopg2.connect(dbname="rdt",user="user",host="host",password=postgres_pw)
    conn.autocommit = True
    cur=conn.cursor()
    print("Database connected")
except:
    print("I am unable to connect to the database")

# The default node, defines the default accessability (aka 'bereikbaarheid')
wellconnectednode=254441013

## Bereikbaarheidsquery
Verkregen van Bas Bussink, Amsterdam.

Aangepast om NDW data te gebruiken.

Aangepast om de verkeersbordensituatie dynamisch te kunnen aanpassen voor een bepaalde wegwerkzaamheid (gekoppeld via MelvinID)

In [2]:
sql='''DROP FUNCTION IF EXISTS prd_conflicten.ST_Bereikbaarheid;
CREATE FUNCTION prd_conflicten.ST_Bereikbaarheid (
    vehicle VARCHAR,
    aanhanger BOOLEAN,
    l FLOAT,
    b FLOAT,
    h FLOAT,
    aslast FLOAT,
    totaalgewicht FLOAT,
    startpoint INT,
    melvinid INT
)
RETURNS TABLE(id int, statuscode INT, geom GEOMETRY) AS $$
    select v.id,
        v.bereikbaar_status_code,
        v.geom 
    from (
        select
            abs(n.id) as id,
            max(case
                when n.cost is NULL then 333  --Eenrichtingsverkeer?
                when routing.agg_cost is null then 222 --Onbereikbaar via andere links?
                -- Wegvak niet begaanbaar voor dit voertuig?
                when 
                    -- verbod voor dit voertuigtype
                    CASE
                        WHEN vehicle = 'auto' AND n.c06 > 0 THEN TRUE
                        WHEN vehicle = 'vrachtauto' AND n.c07 > 0 OR n.c07b > 0 THEN TRUE
                        WHEN vehicle = 'bus' AND n.c07a > 0 OR n.c07b > 0 THEN TRUE
                        WHEN vehicle = 'trekker' AND n.c08 > 0 OR n.c09 > 0 THEN TRUE
                        ELSE false
                    END = true
                    or n.c10 > 0 and aanhanger = true
                    or n.c01 is not null
                    or n.c17 < l
                    or n.c18 < b
                    or n.c19 < h
                    or n.c20 < totaalgewicht
                    or n.c21 < aslast
                then 222
                else 999
            end
        ) as bereikbaar_status_code,
            g.geom as geom
        -- In plaats van de gewone bebording tabel, gebruik een aangepaste tabel
        -- from prd_ndw_borden.nwb_wegvakken_bebording AS n
        from (
            SELECT id,source,target,spd,cost,c01,c06,c07,c07a,c07b,c08,c09,c10,c11,c12,c17::float,c18::float,c19::float,c20::float,c21::float,geom
            FROM prd_ndw_borden.nwb_wegvakken_bebording
            -- Negeer wegsegmenten met werkzaamheden
            WHERE id NOT IN (
                SELECT id
                FROM int_conflicten.ndw_wegvakken_werkzaamheden
                WHERE melvin_id IN (melvinid)
            )
            -- Voeg de nieuwe/tijdelijke situatie van wegvakken met werkzaamheden toe
            UNION
            SELECT id,source,target,spd,cost,c01,c06,c07,c07a,c07b,c08,c09,c10,c11,c12,c17::float,c18::float,c19::float,c20::float,c21::float,geom
            FROM int_conflicten.ndw_wegvakken_werkzaamheden
            WHERE melvin_id IN (melvinid)
        ) AS n
        left join (
            SELECT start_vid as source,
                end_vid as target,
                agg_cost
            FROM pgr_dijkstraCost('
                select id, source, target, cost
                from (
                    SELECT id,source,target,spd,cost,c01,c06,c07,c07a,c07b,c08,c09,c10,c11,c12,c17,c18,c19,c20,c21,geom
                    FROM prd_ndw_borden.nwb_wegvakken_bebording
                    WHERE id NOT IN (
                        SELECT id
                        FROM int_conflicten.ndw_wegvakken_werkzaamheden
                        WHERE melvin_id IN ('||melvinid||')
                    ) 
                    UNION
                    SELECT id,source,target,spd,cost,c01,c06,c07,c07a,c07b,c08,c09,c10,c11,c12,c17,c18,c19,c20,c21,geom
                    FROM int_conflicten.ndw_wegvakken_werkzaamheden
                    WHERE melvin_id IN ('||melvinid||')
                ) as borden
                where cost > 0
                and (
                    (( -.01 + '||l||' ) < c17 or c17 is null)
                    and (( -.01 + '||b||' ) < c18 or c18 is null)
                    and (( -.01 + '||h||' ) < c19 or c19 is null)
                    and (( -1 + '||totaalgewicht||' ) < c20 or c20 is null)
                    and (( -1 + '||aslast||' ) < c21 or c21 is null)
                    and ((c01 < 1) or c01 is null)
                    -- Voertuigbeperking
                    and (
                        -- auto
                        c06 < 1
                        or '||vehicle||' != ''auto''
                    )
                    and (
                        -- vrachtwagen
                        (c07 < 1 and c07b < 1)
                        or '||vehicle||' != ''vrachtwagen''
                        or '||totaalgewicht||' <= 3500
                    )
                    and (
                        -- bus
                        (c07a < 1 and c07b < 1)
                        or '||vehicle||' != ''bus''
                    )
                    and (
                        -- trekker
                        (c08 < 1 and c09 < 1)
                        or '||vehicle||' != ''trekker''
                    )
                    and (
                        -- aanhanger
                        c10 < 1
                        or '||aanhanger||' is false
                    )
                )',
                startPoint,
                array(
                    select node
                    from prd_ndw_wegen.nwb_wegvakken_nodes
                )
            ) as pgr_cost
        ) as routing on n.source = routing.target

        left join prd_ndw_borden.nwb_wegvakken_bebording AS g
            on abs(n.id) = g.id
            where abs(n.id) in (
                select id from prd_ndw_borden.nwb_wegvakken_bebording
                where id > 0
            )
            and n.cost > 0

        group by abs(n.id), g.geom
        order by abs(n.id)
    ) AS v
    where v.bereikbaar_status_code <> 999;
$$ LANGUAGE SQL;'''
cur.execute(sql)

## Onbereikbaarheid per geisoleerde wegwerkzaamheid
De tabel vehicles bevat een aantal standaardvoertuigen die de standaardruimte aan verschillen in voertuigen zo goed mogelijk afdekken. De (on)bereikbaarheid als gevolg van een wegafzetting wordt afzonderlijk voor elk van deze voertuigen doorgerekend.

De _aanvullende_ (on)bereikbaarheid per wegwerkzaamheid wordt opgeslagen. De totale onbereikbaarheid per wegwerkzaamheid voor een specifiek voertuig is te achterhalen als de union van de onbereikbaarheid per wegwerkzaamheid en de standaard onbereikbaarheid voor dat voertuig (uit de tabel vehicles_default_bereikbaarheid).

In [3]:
sql=f'''DROP TABLE IF EXISTS int_conflicten.vehicles_default_bereikbaarheid;
CREATE TABLE int_conflicten.vehicles_default_bereikbaarheid AS (
    SELECT veh.id AS vehicle_id, roads.*
    FROM stg_conflicten.vehicles AS veh,
    LATERAL (
        SELECT default_full.id
        FROM prd_conflicten.ST_Bereikbaarheid(veh.v_type,veh.trailer,veh.v_length::float,veh.v_width::float,veh.v_height::float,veh.v_axleload::float,veh.v_weight::float,{wellconnectednode},000000) as default_full --intentionally non-matching ID
    ) AS roads
);'''
cur.execute(sql)

In [4]:
# Output: additionele wegsegmenten die onbereikbaar worden vanwege wegwerkzaamheid
# Runs for all rows in the vehicles table
sql='''DROP FUNCTION IF EXISTS ST_Bereikbaarheid_vehicle_melvinid;
CREATE FUNCTION ST_Bereikbaarheid_vehicle_melvinid (
    startpoint INT, -- DEFAULT 254441013
    melvinid INT
)
RETURNS TABLE(vehicleid INT, roadid int, statuscode INT, geom GEOMETRY) AS $$
    SELECT veh.id, roads.*
    FROM stg_conflicten.vehicles AS veh,
    -- CROSS JOIN LATERAL (
    LATERAL (
        SELECT *
        FROM ST_Bereikbaarheid(veh.v_type,veh.trailer,veh.v_length,veh.v_width,veh.v_height,veh.v_axleload,veh.v_weight,startpoint,melvinid) as roadworks_full 
        WHERE roadworks_full.id NOT IN (
            SELECT default_full.id
            FROM int_conflicten.vehicles_default_bereikbaarheid as default_full
            WHERE default_full.vehicle_id = veh.id
        )
    ) AS roads;
$$ LANGUAGE SQL;'''
cur.execute(sql)

In [5]:
# sql=f'''DROP TABLE IF EXISTS int_conflicten.onbereikbaar_melvinid;
# CREATE TABLE int_conflicten.onbereikbaar_melvinid AS (
#     SELECT melvin.melvin_id, onbereikbaarheid.*
#     from (
#         --Als er meerdere wegsegmenten in 1x worden afgesloten, staat dezelfde melvin_id meerdere keren in de tabel
#         SELECT DISTINCT melvin_id
#         FROM int_conflicten.ndw_wegvakken_werkzaamheden
#     ) AS melvin,
#     LATERAL ST_Bereikbaarheid_vehicle_melvinid({wellconnectednode},melvin.melvin_id) AS onbereikbaarheid
# );'''
# cur.execute(sql)

In [26]:
sql_lookup=f'''
SELECT DISTINCT melvin_id
FROM int_conflicten.ndw_wegvakken_werkzaamheden
ORDER BY melvin_id'''
cur.execute(sql_lookup)
data=cur.fetchall()
melvin_ids = [item for sublist in data for item in sublist]

sql_create=f'''DROP TABLE IF EXISTS int_conflicten.onbereikbaar_melvinid;
CREATE TABLE int_conflicten.onbereikbaar_melvinid (
    melvin_id INT,
	vehicleid INT,
    roadid int,
    statuscode INT,
    geom GEOMETRY
);'''
cur.execute(sql_create)

for melvin in melvin_ids:
    # print(melvin)
    sql_insert=f'''INSERT INTO int_conflicten.onbereikbaar_melvinid(melvin_id,vehicleid,roadid,statuscode,geom)
        SELECT {melvin} as melvin_id, onbereikbaarheid.*
        FROM ST_Bereikbaarheid_vehicle_melvinid({wellconnectednode},{melvin}) AS onbereikbaarheid'''
    cur.execute(sql_insert)

## Conflicten
Conflicten ontstaan als meerdere wegwerkzaamheden samen een groter effect op de bereikbaarheid hebben dan de wegwerkzaamheden afzonderlijk

Dit is bijna volledig een kopie van ST_bereikbaarheid, maar gebruikt een conflictid ipv melvinid.

Het is netter als deze 2 functies nog samengevoegd worden

In [ ]:
# conflictid wordt gebruikt om (via tabel int_conflicten.conflict_potential) op te zoeken welke melvinid bij welk conflictid horen en zo de bereikbaarheid van gekoppelde wegwerkzaamheden te berekenen
sql='''DROP FUNCTION IF EXISTS ST_Bereikbaarheid_conflict;
CREATE FUNCTION ST_Bereikbaarheid_conflict (
    vehicle VARCHAR,
    aanhanger BOOLEAN,
    l FLOAT,
    b FLOAT,
    h FLOAT,
    aslast FLOAT,
    totaalgewicht FLOAT,
    startpoint INT,
    conflictid INT
)
RETURNS TABLE(id int, statuscode INT, geom GEOMETRY) AS $$
    select v.id,
        v.bereikbaar_status_code,
        v.geom 
    from (
        select
            abs(n.id) as id,
            max(case
                when n.cost is NULL then 333  --Eenrichtingsverkeer?
                when routing.agg_cost is null then 222 --Onbereikbaar via andere links?
                -- Wegvak niet begaanbaar voor dit voertuig?
                when 
                    -- verbod voor dit voertuigtype
                    CASE
                        WHEN vehicle = 'auto' AND n.c06 > 0 THEN TRUE
                        WHEN vehicle = 'vrachtauto' AND n.c07 > 0 OR n.c07b > 0 THEN TRUE
                        WHEN vehicle = 'bus' AND n.c07a > 0 OR n.c07b > 0 THEN TRUE
                        WHEN vehicle = 'trekker' AND n.c08 > 0 OR n.c09 > 0 THEN TRUE
                        ELSE false
                    END = true
                    or n.c10 > 0 and aanhanger = true
                    or n.c01 is not null
                    or n.c17 < l
                    or n.c18 < b
                    or n.c19 < h
                    or n.c20 < totaalgewicht
                    or n.c21 < aslast
                then 222
                else 999
            end
        ) as bereikbaar_status_code,
            g.geom as geom
        -- In plaats van de gewone bebording tabel, gebruik een aangepaste tabel
        -- from prd_ndw_borden.nwb_wegvakken_bebording AS n
        from (
            SELECT id,source,target,spd,cost,c01,c06,c07,c07a,c07b,c08,c09,c10,c11,c12,c17,c18,c19,c20,c21,geom
            FROM prd_ndw_borden.nwb_wegvakken_bebording
            -- Negeer wegsegmenten met werkzaamheden
            WHERE id NOT IN (
                SELECT id
                FROM int_conflicten.ndw_wegvakken_werkzaamheden
                WHERE melvin_id IN (SELECT melvin_id::int FROM int_conflicten.conflict_potential AS cp WHERE cp.conflict_id=conflictid)
            )
            -- Voeg de nieuwe/tijdelijke situatie van wegvakken met werkzaamheden toe
            UNION
            SELECT id,source,target,spd,cost,c01,c06,c07,c07a,c07b,c08,c09,c10,c11,c12,c17,c18,c19,c20,c21,geom
            FROM int_conflicten.ndw_wegvakken_werkzaamheden
            WHERE melvin_id IN (SELECT melvin_id::int FROM int_conflicten.conflict_potential AS cp WHERE cp.conflict_id=conflictid)
        ) AS n
        left join (
            SELECT start_vid as source,
                end_vid as target,
                agg_cost
            FROM pgr_dijkstraCost('
                select id, source, target, cost
                from (
                    SELECT id,source,target,spd,cost,c01,c06,c07,c07a,c07b,c08,c09,c10,c11,c12,c17,c18,c19,c20,c21,geom
                    FROM prd_ndw_borden.nwb_wegvakken_bebording
                    WHERE id NOT IN (
                        SELECT id
                        FROM int_conflicten.ndw_wegvakken_werkzaamheden
                        WHERE melvin_id IN (SELECT melvin_id::int FROM int_conflicten.conflict_potential AS cp WHERE cp.conflict_id='||conflictid||')
                    ) 
                    UNION
                    SELECT id,source,target,spd,cost,c01,c06,c07,c07a,c07b,c08,c09,c10,c11,c12,c17,c18,c19,c20,c21,geom
                    FROM int_conflicten.ndw_wegvakken_werkzaamheden
                    WHERE melvin_id IN (SELECT melvin_id::int FROM int_conflicten.conflict_potential AS cp WHERE cp.conflict_id='||conflictid||')
                ) as borden
                where cost > 0
                and (
                    (( -.01 + '||l||' ) < c17 or c17 is null)
                    and (( -.01 + '||b||' ) < c18 or c18 is null)
                    and (( -.01 + '||h||' ) < c19 or c19 is null)
                    and (( -1 + '||totaalgewicht||' ) < c20 or c20 is null)
                    and (( -1 + '||aslast||' ) < c21 or c21 is null)
                    and ((c01 < 1) or c01 is null)
                    -- Voertuigbeperking
                    and (
                        -- auto
                        c06 < 1
                        or '||vehicle||' != ''auto''
                    )
                    and (
                        -- vrachtwagen
                        (c07 < 1 and c07b < 1)
                        or '||vehicle||' != ''vrachtwagen''
                        or '||totaalgewicht||' <= 3500
                    )
                    and (
                        -- bus
                        (c07a < 1 and c07b < 1)
                        or '||vehicle||' != ''bus''
                    )
                    and (
                        -- trekker
                        (c08 < 1 and c09 < 1)
                        or '||vehicle||' != ''trekker''
                    )
                    and (
                        -- aanhanger
                        c10 < 1
                        or '||aanhanger||' is false
                    )
                )',
                startPoint,
                array(
                    select node
                    from prd_ndw_wegen.nwb_wegvakken_nodes
                )
            ) as pgr_cost
        ) as routing on n.source = routing.target

        left join prd_ndw_borden.nwb_wegvakken_bebording AS g
            on abs(n.id) = g.id
            where abs(n.id) in (
                select id from prd_ndw_borden.nwb_wegvakken_bebording
                where id > 0
            )
            and n.cost > 0

        group by abs(n.id), g.geom
        order by abs(n.id)
    ) AS v
    where v.bereikbaar_status_code <> 999;
$$ LANGUAGE SQL;'''
cur.execute(sql)

In [ ]:
# Output: additionele wegsegmenten die onbereikbaar worden vanwege _combinatie_ van wegwerkzaamheden (aangeduidt met een conflictid)
# Runs for all rows in the vehicles table
sql='''DROP FUNCTION IF EXISTS ST_Bereikbaarheid_vehicle_conflictid;
CREATE FUNCTION ST_Bereikbaarheid_vehicle_conflictid (
    startpoint INT, -- DEFAULT 254441013
    conflictid INT
)
RETURNS TABLE(vehicleid INT, roadid int, statuscode INT, geom GEOMETRY) AS $$
    SELECT veh.id as vehicle, roads.*
    FROM stg_conflicten.vehicles AS veh,
    LATERAL (
        SELECT *
        FROM ST_Bereikbaarheid_conflict(veh.v_type,veh.trailer,veh.v_length,veh.v_width,veh.v_height,veh.v_axleload,veh.v_weight,startpoint,conflictid) as roadworks_full 
        WHERE roadworks_full.id NOT IN (
            -- onbereikbaar voor dit voertuig ongeacht wegwerkzaamheid
            SELECT default_full.id::int
            FROM int_conflicten.vehicles_default_bereikbaarheid as default_full
            WHERE default_full.vehicle_id = veh.id
            UNION
            -- onbereikbaar door 1 v/d losse wegwerkzaamheden
            SELECT roadid::int
            FROM int_conflicten.onbereikbaar_melvinid as individual_roadworks
            WHERE (melvin_id::int) IN (SELECT melvin_id::int FROM int_conflicten.conflict_potential AS cp WHERE cp.conflict_id=conflictid)
                and individual_roadworks.vehicleid = veh.id
        )
    ) AS roads;
$$ LANGUAGE SQL;'''
cur.execute(sql)

## Potentiele conflicten

Wegwerkzaamheden intelligent koppelen als potentieel conflictueus: als ze zowel in ruimte als tijd in elkaars buurt liggen

In [ ]:
sql='''-- create timeintervals in which to check for conflicts
DROP TABLE IF EXISTS int_conflicten.intervaltable;
CREATE TABLE int_conflicten.intervaltable AS (
    SELECT time, 
        CASE 
            WHEN (SELECT count(*) FROM int_conflicten.onbereikbaar_conflictid_history) = 0 THEN row_number() OVER(ORDER BY time)::INT
            ELSE (SELECT MAX(conflict_id) FROM int_conflicten.onbereikbaar_conflictid_history)+row_number() OVER(ORDER BY time)::INT
        END AS interval_id
    FROM (
        SELECT (timeperiod_startDate) as time 
        FROM int_conflicten.ndw_werkzaamheden_wvid_cleaned 
        UNION 
        SELECT (timeperiod_endDate+'7 days'::interval) as time 
        FROM int_conflicten.ndw_werkzaamheden_wvid_cleaned
    ) as t
);'''
cur.execute(sql)

In [ ]:
sql='''-- join intervals to the melvin_ids that occupy that interval
DROP TABLE IF EXISTS int_conflicten.conflict_potential;
CREATE TABLE int_conflicten.conflict_potential AS (
    SELECT 
        i.time AS startDate
        ,j.time as endDate
        ,i.interval_id AS conflict_id
        ,melvin.melvin_id
    FROM int_conflicten.intervaltable AS i
    ,LATERAL (
        SELECT time
        FROM int_conflicten.intervaltable AS jt
        WHERE jt.time>i.time 
        ORDER BY jt.time ASC 
        LIMIT 1
    ) AS j
    ,LATERAL (
        SELECT DISTINCT melvin_id 
        FROM int_conflicten.ndw_werkzaamheden_wvid_cleaned  
        WHERE (timeperiod_startDate)<=j.time
        and (timeperiod_endDate+'7 days'::interval)>=i.time
    ) as melvin
    ORDER BY i.time
);'''
cur.execute(sql)

In [ ]:
# # for each interval, run the ST_Bereikbaarheid_vehicle_conflictid
# # create table containing all onbereikbaar links, for all vehicle types, for all roadworks/melvinIDs
# sql=f'''DROP TABLE IF EXISTS int_conflicten.onbereikbaar_conflictid;
# CREATE TABLE int_conflicten.onbereikbaar_conflictid AS (
#     SELECT cp.conflict_id::int, onbereikbaarheid.*
#     from (
#         SELECT DISTINCT conflict_id::int
#         FROM int_conflicten.conflict_potential
#     ) AS cp,
#     LATERAL ST_Bereikbaarheid_vehicle_conflictid({wellconnectednode},cp.conflict_id::INT) AS onbereikbaarheid
# );'''
# cur.execute(sql)

In [ ]:
sql_lookup=f'''
SELECT DISTINCT conflict_id::int
FROM int_conflicten.conflict_potential'''
cur.execute(sql_lookup)
data=cur.fetchall()
conflict_ids = [item for sublist in data for item in sublist]

sql_create=f'''DROP TABLE IF EXISTS int_conflicten.onbereikbaar_conflictid;
CREATE TABLE int_conflicten.onbereikbaar_conflictid (
    conflict_id INT,
	vehicleid INT,
    roadid int,
    statuscode INT,
    geom GEOMETRY
);'''
cur.execute(sql_create)

for conflict in conflict_ids:
    print(conflict)
    sql_insert=f'''INSERT INTO int_conflicten.onbereikbaar_conflictid(conflict_id,vehicleid,roadid,statuscode,geom)
        SELECT {conflict} as conflict_id, onbereikbaarheid.*
        FROM ST_Bereikbaarheid_vehicle_conflictid({wellconnectednode},{conflict}) AS onbereikbaarheid'''
    cur.execute(sql_insert)

In [ ]:
# # Compare onbereikbaar_conflictid to known conflicts
# # Ignore cases of partical overlap for now
# sql='''DROP FUNCTION IF EXISTS ST_check_history;
# CREATE FUNCTION ST_check_history(c_id INT, vehicle INT) RETURNS void AS $$
#     DECLARE
#         var_melvin_id_array INT[] := ARRAY(
#             SELECT melvin_id
#             FROM int_conflicten.conflict_potential AS cp
#             WHERE cp.conflict_id=c_id
#         );
#         var_c_id INT;
#         var_time timestamp;
#     BEGIN
#         SELECT cp.startDate
#         FROM int_conflicten.conflict_potential AS cp
#         INTO var_time
#         WHERE cp.conflict_id=c_id;

#         SELECT och.conflict_id
#         FROM int_conflicten.onbereikbaar_conflictid_history AS och
#         INTO var_c_id
#         WHERE och.melvin_id_array = var_melvin_id_array;

#         if not found then
#             INSERT INTO int_conflicten.onbereikbaar_conflictid_history
#             VALUES(var_time,c_id,var_melvin_id_array,vehicle,'new');
#         -- else
#         --     raise notice '';
#         end if;
#     END;
# $$ LANGUAGE plpgsql;'''
# cur.execute(sql)

In [ ]:
# # Compare onbereikbaar_conflictid to known conflicts
# # Ignore cases of partical overlap for now
# sql='''DROP FUNCTION IF EXISTS ST_check_current;
# CREATE FUNCTION ST_check_current(conflict_id INT,var_melvin_id_array INT[]) RETURNS void AS $$
#     DECLARE
#         var_melvin_id_array INT[] := ARRAY(
#             SELECT melvin_id
#             FROM int_conflicten.conflict_potential AS cp
#             WHERE cp.conflict_id=conflict_id
#         );
#         var_conflict_id INT;
#         var_time timestamp;
#     BEGIN
#         -- make Temp table for melvin_id_array lookup
#         DROP TABLE IF EXISTS int_conflicten.conflict_potential_array_tempTable;
#         CREATE TABLE int_conflicten.conflict_potential_array_tempTable AS (
#             SELECT cp.conflict_id, 
#                 ARRAY(
#                     SELECT melvin_id
#                     FROM int_conflicten.conflict_potential AS cp2
#                     WHERE cp2.conflict_id=cp.conflict_id
#                 ) as melvin_id_array
#             from (
#                 SELECT DISTINCT cp3.conflict_id
#                 FROM int_conflicten.conflict_potential as cp3
#             ) AS cp
#         );

#         SELECT och.conflict_id
#         FROM int_conflicten.conflict_potential_array_tempTable AS oc
#         INTO var_conflict_id
#         WHERE oc.melvin_id_array = var_melvin_id_array;

#         if not found then
#             UPDATE int_conflicten.onbereikbaar_conflictid_history as hist
#             SET hist.statuscode = 'solved in source (Melvin)'
#             WHERE hist.conflict_id = conflict_id;
#         -- else
#         --     raise notice '';
#         end if;
#     END;
# $$ LANGUAGE plpgsql;'''
# cur.execute(sql)

## Queries to display results in WebApp

In [ ]:
# Seperate roadworks
sql='''DROP TABLE IF EXISTS prd_conflicten.displayTable_melvin;
CREATE TABLE prd_conflicten.displayTable_melvin AS (
    SELECT om.melvin_id AS id, v.v_type as prototype_vehicle, roadid, m.city, m.roadname, m.roadauthority, s.source, m.timeperiod_startDate AS startdate, m.timeperiod_endDate as enddate, geom
    FROM int_conflicten.onbereikbaar_melvinid AS om
    LEFT JOIN stg_conflicten.vehicles AS v
    ON om.vehicleid=v.id
    -- add time information
    LEFT JOIN (
        SELECT *, row_number() over (partition by melvin_id order by road_id) as rn
        FROM int_conflicten.ndw_werkzaamheden_wvid_cleaned
    ) AS m
    ON om.melvin_id=(m.melvin_id::int)
    AND m.rn=1
    -- source column
    LEFT JOIN (
        SELECT road_id, 'melvin' AS source
        from int_conflicten.ndw_werkzaamheden_wvid_cleaned AS m2
        UNION
        SELECT roadid as road_id, 'query' AS source
        FROM int_conflicten.onbereikbaar_melvinid AS om2
        WHERE om2.roadid NOT IN (SELECT road_id FROM int_conflicten.ndw_werkzaamheden_wvid_cleaned)
    ) as s
    ON roadid = s.road_id
    ORDER BY m.timeperiod_startDate, om.melvin_id, om.roadid, v.v_type
);'''
cur.execute(sql)

In [ ]:
# conflicts
sql='''DROP TABLE IF EXISTS prd_conflicten.displayTable_conflict;
CREATE TABLE prd_conflicten.displayTable_conflict AS (
    SELECT DISTINCT oc.conflict_id AS id, oc.roadid, w.geom
                    ,cp.source, cp.startdate, cp.enddate
                    ,v.v_type as prototype_vehicle
                    ,ws.wvk_id as road_id, ws.wegbehnaam as roadauthority
    FROM int_conflicten.onbereikbaar_conflictid AS oc
    LEFT JOIN stg_conflicten.vehicles AS v
    ON oc.vehicleid=v.id
    LEFT JOIN (
        SELECT cp1.conflict_id, cp1.startdate, cp1.enddate, 'conflict' AS source, row_number() over (partition by conflict_id order by melvin_id) as rn
        FROM int_conflicten.conflict_potential as cp1
        UNION
        (
            SELECT cp2.conflict_id, cp2.startdate, cp2.enddate, 'component' AS source, 1 as rn
            FROM int_conflicten.conflict_potential as cp2
        )
    ) AS cp
    ON oc.conflict_id=cp.conflict_id AND cp.rn=1
    LEFT JOIN prd_ndw_borden.nwb_wegvakken_bebording as w
    ON oc.roadid=w.id
    LEFT JOIN stg_ndw_wegen.nwb_raw_subselection as ws
    ON oc.roadid=ws.wvk_id
    ORDER BY cp.startdate, oc.conflict_id, oc.roadid, prototype_vehicle
);'''
cur.execute(sql)

In [ ]:
# status
sql='''DROP TABLE IF EXISTS prd_conflicten.displayTable_conflict_status;
CREATE TABLE prd_conflicten.displayTable_conflict_status AS (
    SELECT statuscode, count(*)
    FROM int_conflicten.onbereikbaar_conflictid_history
    GROUP BY statuscode
);'''
cur.execute(sql)